<a href="https://colab.research.google.com/github/ibwi/ibwi/blob/main/Recommandation_POIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommandation de points d’intérêts sur les réseaux sociaux

- Auteur : Hy-Boui CHANG
- Classe : Executive Master IASD 2019-2020 - promo 1


## 1. Introduction

La source des données se trouve sous l'adresse suivante : https://drive.google.com/ﬁle/d/1DBq98KRWGRQyAJ0TqBqBKETGT90cyLwh/view?usp=sharing

Nous récupérons 4 dataframes que nous chargeons dans ce notebook.

Note : 
- **checkins_count** représente le nombre de visites faites pour ce point d'intérêt (POI)
- **user_count** représente le nombre de visiteurs différents (on a donc user_count < checkins_count, puisqu'un visiteur peut visiter plusieurs fois un POI)
- **chekins_num** représente le nombre de visites faites par le visiteur

liens utiles :
- https://blog.dominodatalab.com/recommender-systems-collaborative-filtering/
- http://www.python-simple.com/python-pandas/dataframes-indexation.php
- https://www.geeksforgeeks.org/create-a-list-from-rows-in-pandas-dataframe/
- https://www.it-swarm.dev/fr/python/compter-la-frequence-laquelle-une-valeur-apparait-dans-une-colonne-dataframe/1044993957/
- https://stackoverflow.com/questions/17679089/pandas-dataframe-groupby-two-columns-and-get-counts
- https://riptutorial.com/fr/pandas/example/5958/ajouter-une-nouvelle-colonne
- https://surprise.readthedocs.io/en/stable/getting_started.html#train-test-split-example
- https://surprise.readthedocs.io/en/stable/getting_started.html#train-test-split-example
- https://github.com/NicolasHug/Surprise/issues/193
- https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Building%20Recommender%20System%20with%20Surprise.ipynb
- https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b
- https://stackoverrun.com/fr/q/12211705
- http://forums.cirad.fr/logiciel-r/viewtopic.php?t=9205
- https://www.it-swarm.dev/fr/python/formule-de-distance-entre-deux-points-dune-liste/971630733/
- https://gist.github.com/rochacbruno/2883505
- https://www.it-swarm.dev/fr/python/generer-toutes-les-combinaisons-dune-liste-en-python/1073071318
- https://stackoverrun.com/fr/q/12211705
- https://stackoverflow.com/questions/9078776/python-module-for-storing-and-querying-geographical-coordinates
- https://surprise.readthedocs.io/en/stable/building_custom_algo.html
- https://www.delftstack.com/fr/howto/python-pandas/how-to-get-a-value-from-a-cell-of-a-dataframe/#il-est-possible-dobtenir-la-valeur-dune-cellule-dune-dataframe-de-pandas-en-utilisant-iloc
- https://r.search.yahoo.com/_ylt=AwrJS9TqEIZfxXcA2R0k24lQ;_ylu=Y29sbwNpcjIEcG9zAzMEdnRpZANCOTI3N18xBHNlYwNzcg--/RV=2/RE=1602650474/RO=10/RU=https%3a%2f%2fwww.mdpi.com%2f2220-9964%2f9%2f4%2f285%2fpdf/RK=2/RS=Cm4YLEReCSlR_m31c8ctGHbpAwM-
- # https://surprise.readthedocs.io/en/stable/FAQ.html (precision et recall)



In [ ]:
# chargement des librairies

from tqdm import tqdm
import numpy as np
import pandas as pd

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 

from surprise import NormalPredictor, SVD, KNNBasic, NMF, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from surprise.model_selection import cross_validate, KFold

from surprise.accuracy import rmse
from surprise import accuracy

from pyproj import Geod

import statistics
from math import exp, sqrt, pow, pi

from surprise import AlgoBase
from surprise import PredictionImpossible


## 2. Chargement des données et traitement

**2.1. Chargement des dataframes**

Nous allons charger les 4 dataframes :
- checkins qui associe à chaque user id, un POI id qu'il a visité. S'il a visité plusieurs POIs, cela se reflète dans le dataframe par plusieurs lignes
- friendship qui associe un userid avec un userid ami. Idem, si ce user id a plusieurs amis, cela se reflète dans le dataframe par plusieurs lignes
- locations qui donne l'id du POI, sa longitude, sa latitude, le nombre de visites reçus, le nombre de visiteurs différents reçus
- user info qui donne pour un user id donné, son nombre d'amis et ainsi que le nombre de visites effectuées.


In [ ]:
# 2.1_Chargement des données dans des dataframes Pandas

CHECKINS_DATA_FILE = './data/raw/gowalla_checkins.csv'
FRIENDSHIP_DATA_FILE = './data/raw/gowalla_friendship.csv'
LOCATIONS_DATA_FILE = './data/raw/gowalla_locations.csv'
USERINFO_DATA_FILE = './data/raw/gowalla_userinfo.csv'

# load the raw csv into a data_frame
df_checkins = pd.read_csv(CHECKINS_DATA_FILE)
df_friendship = pd.read_csv(FRIENDSHIP_DATA_FILE)
df_locations = pd.read_csv(LOCATIONS_DATA_FILE)
df_userinfo = pd.read_csv(USERINFO_DATA_FILE)



Pour la suite, nous allons volontairement restreindre la taille des dataframes.
Ceci est rendu nécessaire pour lancer les codes qui suivent sur l'ordinateur que je possède.


In [ ]:
# réduction du dataframe "checkins" en ne sélectionnant que les userid qui sont entre 1 et 100 inclus

print("concernant le dataframe checkins entier")
print(df_checkins.head(5))
print(df_checkins.shape)
print(df_checkins.min())

print("\n concernant le dataframe checkins réduit aux userid inférieurs ou égales à 100")
#df_checkinsbis = df_checkins.sort_values(by='userid')
df_checkins2=df_checkins.loc[df_checkins['userid'] <= 100]
print(df_checkins2.shape)
print(df_checkins2.head(5))
print(df_checkins2.min())
print(df_checkins2.max())

# idée finalement abandonnée :
#df_checkins2 = df_checkins.loc[:9999,:]


concernant le dataframe checkins entier
   userid  placeid
0    1338   482954
1    1338   580963
2    1338   365256
3    1338    89504
4    1338  1267135
(36001959, 2)
userid        1
placeid    8904
dtype: int64

 concernant le dataframe checkins réduit aux userid inférieurs ou égales à 100
(109746, 2)
       userid  placeid
75932      65    34812
75933      65   281276
75934      65   185449
75935      65  1253901
75936      65    11703
userid        1
placeid    8904
dtype: int64
userid         100
placeid    7569147
dtype: int64


In [ ]:
# réduction du dataframe "friendship" en ne sélectionnant que les userid1 et userid2 qui sont entre 1 et 100 inclus

print(df_friendship.head(5))
print(df_friendship.shape)

df_friendship2=df_friendship.loc[df_friendship['userid1'] <= 100]
df_friendship2=df_friendship2.loc[df_friendship['userid2'] <= 100]
#df_friendship2 = df_friendship.loc[:9999,:]
print(df_friendship2.shape)
print(df_friendship2.head(5))


   userid1  userid2
0        1    63488
1        1        2
2        1        3
3        1        4
4        1        5
(4418339, 2)
(460, 2)
   userid1  userid2
1        1        2
2        1        3
3        1        4
4        1        5
5        1        6


In [ ]:
# jetons un coup d'oeil au dataframe df_locations
# la colonne id représente la référence du POI

print(df_locations.head(5))
df_locations.shape
print(df_locations.min())
# len(df_locations) # on trouve 2 724 891 lignes de données

     id         lng        lat  checkins_count  users_count
0  8904  -94.607499  39.052318             114           21
1  8932  -97.254356  32.927662              67           48
2  8936  -94.591995  39.053318              75           46
3  8938  -94.590311  39.052824             438           94
4  8947 -122.029631  37.331880            3100         1186
id                8904.000000
lng               -176.308594
lat                -90.000000
checkins_count       0.000000
users_count          0.000000
dtype: float64


In [ ]:
# jetons un coup d'oeil au dataframe df_userinfo

print(df_userinfo.head(5))
print(df_userinfo.shape)
#len(df_userinfo) # on trouve 407 533 lignes de données

# réduisons également ici les données pour ne prendre que les userid qui sont entre 1 et 100 inclus
df_userinfo2 =df_userinfo.loc[df_userinfo['id'] <= 100]
print(df_userinfo2.head(5))
print(df_userinfo2.shape)

# on passe donc d'un dataframe de 407 533 lignes, à 72 lignes.

   id  friends_count  checkin_num
0   1            372         1766
1   2            775         2892
2   3            100         3021
3   4            179         1325
4   5            525         3215
(407533, 3)
   id  friends_count  checkin_num
0   1            372         1766
1   2            775         2892
2   3            100         3021
3   4            179         1325
4   5            525         3215
(72, 3)


**2.2. Réduction du dataframe df_user_info**

L'énoncé du TP demande de retirer les users qui ont effectué moins de 5 visites.

Pour la suite, je choisis de réduire aux users qui ont effectué moins de 1000 visites, toujours pour un soucis de réduction de la taille des données à traiter. Le nbre de visites est modulable avec la variable N à modifier. On garde les users qui visitent beaucoup.

Le dataframe de départ compte 407 533 lignes de données :
- après avoir retiré les users de moins de 5 visites, on trouve 243 294 lignes de données
- après avoir retiré les users de moins de 1000 visites, on trouve 5 432 lignes de données

In [ ]:
N=1000
df_userinfo2 = df_userinfo.drop(df_userinfo[df_userinfo.checkin_num<N].index)
len(df_userinfo2)

5432

**2.3. Réduction du dataframe df_locations**

L'énoncé du TP demande de retirer les POIs qui ont reçu moins de 10 visites.

Pour la suite, je choisis de réduire à moins de 1000 visites, toujours pour un soucis de réduction de la taille des données à traiter. Le nbre de visites est modulable avec la variable M à modifier. On garde donc ici les POIs qui sont les plus fréquentés.

Le dataframe de départ compte 2 724 891 lignes de données :
- après avoir retiré les POIs ayant reçu moins de 10 visites, on trouve 831 496 lignes de données
- après avoir retiré les POIs ayant reçu moins de 1000 visites, on trouve 1 148 lignes de données

In [ ]:
M=1000
df_locations2 = df_locations.drop(df_locations[df_locations.checkins_count<M].index)
len(df_locations2)

1148

Affichons maintenant les 50 premières lignes pour pouvoir les utiliser par la suite de notre TP :

In [ ]:
print(df_locations2.head(50))

       id         lng        lat  checkins_count  users_count
4    8947 -122.029631  37.331880            3100         1186
18   8977  -94.714079  39.296746            4525         1718
21   8989  -96.810343  32.790503            3735         1937
25   8995  -96.775185  32.838076            1047          557
45   9056  -97.130454  32.943811            1918          432
52   9066  -96.800001  32.794418            1392          283
58   9073 -122.393725  37.795339            2654         1659
59   9074 -122.407688  37.784698            1161          814
95   9114 -122.407533  37.787954            2672         1789
106  9127 -122.415722  37.808321            1467         1300
110  9131 -122.251911  37.872956            1185          562
123  9153 -122.427177  37.759689            1743          946
131  9175 -122.161424  37.444839            1517          486
134  9181  -77.050166  38.889262            1507         1406
136  9184  -77.035253  38.889459            1757         1515
137  918

**2.4. Création de df_user_friends**

L'énoncé du TP demande de créer un nouveau dataframe qui donne pour chaque user, sa liste d'amis (id).
Ce traitement peut demandes quelques minutes pour être exécuté si les données sont conséquentes.


In [ ]:
df_user_friends = df_friendship2.groupby("userid1")["userid2"].apply(list).to_frame("List_friends").reset_index()
print(df_user_friends.head(5))
print(len(df_user_friends))

   userid1                                       List_friends
0        1  [2, 3, 4, 5, 6, 7, 12, 18, 24, 25, 26, 28, 29,...
1        2  [1, 3, 4, 5, 6, 7, 11, 12, 18, 25, 29, 33, 40,...
2        3  [1, 2, 4, 5, 6, 7, 12, 18, 29, 33, 40, 44, 56,...
3        4  [1, 2, 3, 5, 6, 7, 12, 18, 24, 25, 29, 33, 43,...
4        5  [1, 2, 3, 4, 6, 7, 11, 12, 18, 24, 25, 29, 33,...
58


**2.5. Création de df_user_POI**

L'énoncé du TP demande de créer un nouveau dataframe qui donne pour chaque user, la liste des POIs (id) qu'il a visité.


In [ ]:
df_user_POI = df_checkins2.groupby("userid")["placeid"].apply(list).to_frame("List_POI").reset_index()
print(df_user_POI.head(5))
print(len(df_user_POI.index))

   userid                                           List_POI
0       1  [1440379, 8938, 128291, 8977, 46946, 4713250, ...
1       2  [220372, 17831, 46946, 1538770, 1305312, 20486...
2       3  [7149333, 13252, 2020206, 21714, 536687, 75514...
3       4  [789570, 7540930, 7569147, 6906866, 7493126, 1...
4       5  [43221, 36938, 576585, 28316, 11888, 18268, 18...
72


In [ ]:
# information utile pour plus tard : la liste des POIs du userid 3
listPOI_User3 = df_user_POI.iloc[2]['List_POI'] # df_user_POI['List_POI'].loc[df_user_POI['userid'] == 3]
print(listPOI_User3)



[7149333, 13252, 2020206, 21714, 536687, 75514, 21134, 656248, 9410, 820726, 414070, 2228649, 33472, 7506215, 68676, 9410, 201098, 7149333, 820726, 10577, 75514, 21714, 9371, 21134, 75514, 414070, 414070, 414070, 2020206, 21714, 9371, 420315, 21134, 21714, 9371, 6966378, 420315, 26503, 11392, 7149333, 820726, 853184, 18758, 9225, 21714, 9371, 6696755, 420315, 21714, 9371, 420315, 21714, 420315, 21714, 9371, 15593, 7149333, 9410, 7431624, 7083398, 7431624, 7431624, 7431624, 438445, 285313, 1307020, 159539, 7431624, 136104, 78290, 21714, 9371, 420315, 42950, 9225, 654558, 21714, 18758, 2020206, 2020206, 11352, 420315, 21714, 9371, 78290, 21714, 21714, 9371, 820726, 2228649, 13252, 345459, 2228649, 7149333, 21134, 345459, 21714, 9371, 420315, 724682, 72530, 9411, 21714, 9371, 420315, 49904, 820726, 10741, 201098, 1272080, 420315, 75514, 414070, 2020206, 665804, 665804, 7138991, 72530, 420315, 21714, 9371, 21714, 9371, 201098, 420315, 21714, 9371, 21714, 9371, 420315, 72530, 18758, 10745, 

**2.6. Création de df_frequencies : la fréquence de chaque pair {user ; POI}**


In [ ]:
df_frequencies = df_checkins2.groupby(['userid', 'placeid']).size().sort_values(ascending=False).reset_index(name='freq')
print(df_frequencies.head(20))
print(len(df_frequencies.index))

# Relevés de données pour la vérification du calcul de l'équation 4 (paragraphe 3.3, géo_proba) :
# le userid 3 a visité à minima les POIs suivants : 420315, 21714, 9371, 17208
# le userid 5 a aussi visité le même POI : 21714
# le userid 5 a visité un autre POI : 420315

    userid  placeid  freq
0       33   420315   413
1        3   420315   378
2        7   420315   321
3       97     9961   317
4       29   420315   268
5        3    21714   229
6        5    21714   225
7        3     9371   224
8        6    21714   220
9       71    29021   211
10      85    30181   209
11       5   420315   204
12      29    17208   201
13      12   420315   197
14       5    17208   176
15       3    17208   166
16       2    21714   162
17      97    26995   157
18      28     9191   157
19       4   420315   156
54621


**2.7. Normalisation de la fréquence**

Nous avons besoin de normaliser la fréquence des visites calculée précédemment, de façon à ramener le chiffre dans l'intervalle [0 ; 10].

Pour cela, nous calculons dans un premier temps les valeurs min (fmin) et max (fmax), puis appliquons la formule mathématique fournie.


In [ ]:
print(df_frequencies.head())
print()


fmini = df_frequencies['freq'].min()
print("la valeur minimale de count est ", fmini)

fmaxi = df_frequencies['freq'].max()
print("la valeur maximale de count est ", fmaxi)

calculNormalisation = (lambda x : 10*np.tanh(10*(x-fmini)/(fmaxi-fmini)))
df_frequencies['freqN']=df_frequencies.freq.apply(calculNormalisation)

print("")
print(df_frequencies.head(5))
print(len(df_frequencies.index))

   userid  placeid  freq
0      33   420315   413
1       3   420315   378
2       7   420315   321
3      97     9961   317
4      29   420315   268

la valeur minimale de count est  1
la valeur maximale de count est  413

   userid  placeid  freq      freqN
0      33   420315   413  10.000000
1       3   420315   378  10.000000
2       7   420315   321   9.999996
3      97     9961   317   9.999996
4      29   420315   268   9.999953
54621


**2.8. Chargement dans SURPRISE**

Nous avons ainsi les éléments requis par la librairie SURPRISE pour le chargement.
En effet, nous avons :
- colonne 1 : user id
- colonne 2 : place id (item_id)
- colonne 3 : ratings (si userid a beaucoup visité le POI (fréquence élevée), on prend l'hypothèse qu'il a aimé)

Nous avons besoin de définir un "reader" en spécifiant les paramètres sur freqN (rating scale).


In [ ]:
reader_frequencies = Reader(rating_scale=(0, 10))
df_frequencies_S = Dataset.load_from_df(df_frequencies[['userid', 'placeid', 'freqN']], reader_frequencies)

**2.9a. Apprentissage - Solution 1 : split des données entre training et test set**

In [ ]:
trainset, testset = train_test_split(df_frequencies_S, test_size=.25)


In [ ]:
trainset

trainset.global_mean
trainset.all_items
trainset.all_ratings
trainset.all_users
trainset.rating_scale

(0, 10)

*Remarque 0 :*

Le df_user_POI doit être créée à partir de l’ensemble d’apprentissage, aﬁn d’éviter d’utiliser des données de l’ensemble test dans la phase d’entraînement.
Il est donc nécessaire, si on utilise un split 75/25, de reconvertir le trainset en dataframe, puis recalculer le dataframe df_user_POI depuis le datatrain.


In [ ]:
iterator = trainset.all_ratings()
# hbc .all_reatings() = Generator function to iterate over all ratings

# création des titres de colonne de notre nouveau dataframe
df = pd.DataFrame(columns=['userid', 'placeid', 'freqN'])

i = 0


for (userid, placeid, freqN) in iterator:
    df.loc[i] = [userid, placeid, freqN]
    i = i+1

df.head()

# on recrée le user_POI_bis qui est composé de col1=userid, col2=liste des POIs. On repart de df

df_user_POI_bis = df.drop(columns=['freqN']).groupby("userid")["placeid"].apply(list).to_frame("List_POI_bis").reset_index()
df_user_POI_bis.head()

,userid,List_POI_bis
0,0.0,"[0.0, 9.0, 16.0, 21.0, 45.0, 52.0, 54.0, 57.0,..."
1,1.0,"[1.0, 27.0, 33.0, 49.0, 74.0, 80.0, 87.0, 96.0..."
2,2.0,"[2.0, 253.0, 429.0, 472.0, 481.0, 519.0, 584.0..."
3,3.0,"[3.0, 64.0, 118.0, 137.0, 189.0, 321.0, 331.0,..."
4,4.0,"[4.0, 10.0, 58.0, 59.0, 184.0, 303.0, 314.0, 5..."


Les résultats que j'obtiens ici m'empêche d'aller plus loin, en raison des "zéros" que je trouve dans la liste des POIs.


**2.9b. Apprentissage - Solution 2 : CrossValidation (pas besoin de split)**

Afin de palier au problème précédent (validité de l'entrainement de l'algo), il est possible d'utiliser une méthode de cross validation de l'ensemble des données. On pourrait alors utiliser directement le dataframe créé "df_frequencies_S".



In [ ]:
# test avec l'algorithme SVD

algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, df_frequencies_S, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6674  0.7180  0.6637  0.7366  0.7115  0.6995  0.0289  
MAE (testset)     0.2825  0.2893  0.2793  0.2998  0.2973  0.2896  0.0080  
Fit time          2.06    2.05    2.04    2.05    2.94    2.23    0.36    
Test time         0.05    0.05    0.09    0.05    0.05    0.06    0.02    


{'test_rmse': array([0.66739237, 0.71804423, 0.66372241, 0.73660667, 0.71153667]),
 'test_mae': array([0.28245703, 0.28926186, 0.27931536, 0.2998257 , 0.29727084]),
 'fit_time': (2.055786371231079,
  2.0520877838134766,
  2.044302463531494,
  2.0526154041290283,
  2.941969156265259),
 'test_time': (0.04783177375793457,
  0.047879695892333984,
  0.08877158164978027,
  0.04688739776611328,
  0.04690957069396973)}

In [ ]:
# test avec l'algorithme KNNBasic

algo = KNNBasic()

# Run 5-fold cross-validation and print results
cross_validate(algo, df_frequencies_S, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7647  0.7532  0.7443  0.7897  0.7936  0.7691  0.0196  
MAE (testset)     0.3130  0.3074  0.3162  0.3209  0.3171  0.3149  0.0045  
Fit time          0.01    0.01    0.02    0.02    0.02    0.02    0.00    
Test time         0.09    0.13    0.09    0.09    0.12    0.10    0.02    


{'test_rmse': array([0.76470175, 0.75318218, 0.74425682, 0.78966222, 0.79364494]),
 'test_mae': array([0.31304432, 0.30743912, 0.31623104, 0.32091076, 0.31709175]),
 'fit_time': (0.010968923568725586,
  0.014885425567626953,
  0.016947269439697266,
  0.015959978103637695,
  0.01688075065612793),
 'test_time': (0.0907585620880127,
  0.13463950157165527,
  0.0877373218536377,
  0.08580565452575684,
  0.12466645240783691)}

In [ ]:
# test avec l'algorithme KNNBaseline

algo = KNNBaseline()

# Run 5-fold cross-validation and print results
cross_validate(algo, df_frequencies_S, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7826  0.7311  0.7354  0.7730  0.7817  0.7608  0.0228  
MAE (testset)     0.3107  0.2965  0.3018  0.3075  0.3184  0.3069  0.0075  
Fit time          0.13    0.12    0.13    0.13    0.13    0.13    0.00    
Test time         0.11    0.14    0.11    0.10    0.10    0.11    0.02    


{'test_rmse': array([0.78264975, 0.73109192, 0.73543311, 0.77303667, 0.78172438]),
 'test_mae': array([0.31065475, 0.29645121, 0.30181446, 0.30745471, 0.3183538 ]),
 'fit_time': (0.12986493110656738,
  0.12456512451171875,
  0.1295781135559082,
  0.129652738571167,
  0.1306164264678955),
 'test_time': (0.1068568229675293,
  0.14169526100158691,
  0.10774922370910645,
  0.09877800941467285,
  0.09977078437805176)}

## 3. Influence géographique

**3.1. Calcule de la distance entre chaque pair de POIs**

Nous partons du dataframe "df_user_POI" pour calculer pour chaque utilisateur u les distances dij entre chaque paire
de POIs visités. On dénotera cette liste de distances par Du.

In [ ]:
wgs84_geod = Geod(ellps='WGS84') 

def Distance(lat1,lon1,lat2,lon2): 
    az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct 
    return dist


'''
# test de la fonction

print(df_locations.head(5))

LngA=float(df_locations.query('id==8947')['lng'])
LatA=float(df_locations.query('id==8947')['lat'])
LngB=float(df_locations.query('id==8936')['lng'])
LatB=float(df_locations.query('id==8936')['lat'])
print(Distance(LatA, LngA, LatB, LngB))
print(Distance(37.331880, -122.029631, 39.053318, -94.591995))

'''

"\n# test de la fonction\n\nprint(df_locations.head(5))\n\nLngA=float(df_locations.query('id==8947')['lng'])\nLatA=float(df_locations.query('id==8947')['lat'])\nLngB=float(df_locations.query('id==8936')['lng'])\nLatB=float(df_locations.query('id==8936')['lat'])\nprint(Distance(LatA, LngA, LatB, LngB))\nprint(Distance(37.331880, -122.029631, 39.053318, -94.591995))\n\n"

**3.2, 3.3. Implémenter l’équation 4**

La fonction fonction geo_proba() qui prend en entrée la liste Lu et un POI, et retourne la probabilité de visite de ce POI.


In [ ]:
############### implémentation de la fonction Distance

wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method 

def Distance(lat1,lon1,lat2,lon2): 
    az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct 
    return dist


############### implémentation de la fonction K
def fctK(x):
    return (1/sqrt(2*pi))*exp(-pow(x, 2)/2)


############### implémentation de la fonction geo_proba()
# les données d'entrée sont
# - la liste Lu : liste des POI visités par l'utilisateur u
# - le point POIx : le point pour lequel on calcule la proba que l'utilisateur u va visiter

def geo_proba(Lu, POIx) :
    
    # nous commençons par déterminer les coordonnées de POIx
    qX='id=='+str(POIx)
    lngX = float(df_locations.query(qX)['lng'])
    latX = float(df_locations.query(qX)['lat'])
    
    # il faut commencer à calculer Du
    
    Du = []
    nbPOI=len(Lu)
        
    for indexA in range(nbPOI):
        
        sommeF = 0
        
        POIa = int(Lu[indexA])
        qA='id=='+str(POIa)
        lngA = float(df_locations.query(qA)['lng'])
        latA = float(df_locations.query(qA)['lat'])
        
        for POIb in Lu[indexA+1 : nbPOI]:
            qB='id=='+str(POIb)
            lngB = float(df_locations.query(qB)['lng'])
            latB = float(df_locations.query(qB)['lat'])
            
            #print("\n POIa ", POIa, ": latA ", latA, "et lngA ", lngA)
            #print("POIb ", POIb, ": latB ", latB, "et lngB ", lngB)
            #print(Distance(latA, lngA, latB, lngB))
            
            Du.append(Distance(latA, lngA, latB, lngB))
    
    print("Du : ", Du)
    
    D = len(Du)
    print("D : ", D)
    
    ecartType = statistics.stdev(Du)
    paramh = 1.06 * ecartType * pow(nbPOI, (-1/5))
    print("ecart type de Du :", ecartType)
    print("paramètre h :", paramh)
    

    sommeF = 0
    
    for indexA in range(nbPOI):   # pour chaque point de Lu, f(x, point)
        
               
        POIa = int(Lu[indexA])
        print("\n calcul de f pour le point fixe : ", POIa)
        qA='id=='+str(POIa)
        lngA = float(df_locations.query(qA)['lng'])
        latA = float(df_locations.query(qA)['lat'])
        
        # on va calculer la somme des k
        
        sommeK=0
        
        for d in range(D) :
            
            print("\n", d)
            print("\n et pour la distance suivante : ", Du[d], ",")
            x=(Du[d]-Distance(latX, lngX, latA, lngA))/paramh
            k = fctK(x)
            print("on trouve x :" , x , "et on trouve k :" , k , ".")
            sommeK += k
                    
        F = (1/(D*paramh)) * sommeK
        
        print("d'où au final sommeK : ", sommeK, "et Fij ", F)
    
    sommeF += F
    
    P = (1/nbPOI) * sommeF
    
    print("\n La probabilité pour le point POIx est ", P)
    return P
    

In [ ]:
# test de la fonction geo_proba

# Relevés de données pour la vérification du calcul de l'équation 4 (paragraphe 3.3, géo_proba) :
# le userid 3 a visité à minima les POIs suivants : 9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, 420315, 15245, 18758
# le userid 5 a aussi visité le même POI : 21714
# le userid 5 a visité un autre POI : 420315

# la liste Lu est la liste des POI visités par l'utilisateur u
#Lu = listPOI_User3 # trop long dans les calculs, je préfère mettre une liste limitée sélectionnée pour l'userid 3
Lu = [9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, 420315, 15245, 18758]

POIx = 420315

geo_proba(Lu, POIx)

Du :  [364.1072348590375, 464.23430029528936, 595.0724660273181, 1051.997995877385, 1059.162046827403, 1124.8984168232785, 169.42099473532667, 1140.0487470689638, 1124.8984168232785, 629.2354239512177, 3723.868584535088, 332.0035950255078, 592.5475221681768, 1151.0275805838955, 1153.98358029554, 1185.8754540144741, 515.7815382284228, 1140.597749147637, 1185.8754540144741, 472.7788834473663, 4087.3026250595076, 274.0527881314529, 842.1817635940587, 843.4354647082657, 863.8593886794971, 630.2705167052557, 809.1496856623068, 863.8593886794971, 804.7816678089818, 4044.0109909280864, 568.1904418251492, 569.3827580883359, 593.5037797672012, 733.1785883224403, 560.5257086931276, 593.5037797672012, 1060.632920780966, 3916.2482599750642, 14.18292671796798, 132.5339799632454, 1135.220161389213, 308.49889550216227, 132.5339799632454, 1607.1541420475364, 3688.4293038481796, 118.35143867457997, 1143.9580515909122, 295.3331902843261, 118.35143867457997, 1611.3404042257596, 3701.414248583025, 1221.54

on trouve x : 0.12090013726503748 et on trouve k : 0.3960372704731312 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -1.1673348960535863 et on trouve k : 0.20184122349167422 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -1.4072451626957503 et on trouve k : 0.14821253216626315 .

 54

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : 0.6587463435452913 et on trouve k : 0.32112914824898264 .

 55

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : 3.3587285274008947 et on trouve k : 0.0014166403043333103 .

 56

 et pour la distance suivante :  1257.6979224220986 ,
on trouve x : 0.16613185605690645 et on trouve k : 0.39347473068272226 .

 57

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 0.12090013726503748 et on trouve k : 0.3960372704731312 .

 58

 et pour la distance suivante :  679.0232609634891 ,
on trouve x : -0.5577887272895643 et on trouve k : 0.3414675367161507 .

 59

 et pour l

on trouve x : -0.052438567706328015 et on trouve k : 0.39839415088372687 .

 47

 et pour la distance suivante :  295.3331902843261 ,
on trouve x : -1.114066189504873 et on trouve k : 0.21448611588805255 .

 48

 et pour la distance suivante :  118.35143867457997 ,
on trouve x : -1.3354699270455248 et on trouve k : 0.16354313574964371 .

 49

 et pour la distance suivante :  1611.3404042257596 ,
on trouve x : 0.5322556100418805 et on trouve k : 0.3462526561165785 .

 50

 et pour la distance suivante :  3701.414248583025 ,
on trouve x : 3.1469328672601575 et on trouve k : 0.002821372617311907 .

 51

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 0.04461801317319634 et on trouve k : 0.39854537738635326 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -1.2436170201454273 et on trouve k : 0.18410847084588466 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -1.4835272867875915 et on trouve k : 0.13273970966316084 .

 54

 et pour

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





 20

 et pour la distance suivante :  4087.3026250595076 ,
on trouve x : 4.370733793327046 et on trouve k : 2.8358167351791545e-05 .

 21

 et pour la distance suivante :  274.0527881314529 ,
on trouve x : -0.39963240766867486 et on trouve k : 0.36832426871151785 .

 22

 et pour la distance suivante :  842.1817635940587 ,
on trouve x : 0.3110955483407481 et on trouve k : 0.3800970161064281 .

 23

 et pour la distance suivante :  843.4354647082657 ,
on trouve x : 0.31266392536221854 et on trouve k : 0.3799111390107755 .

 24

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : 0.33821420419801534 et on trouve k : 0.37676525277261896 .

 25

 et pour la distance suivante :  630.2705167052557 ,
on trouve x : 0.04599509780651832 et on trouve k : 0.39852051253279414 .

 26

 et pour la distance suivante :  809.1496856623068 ,
on trouve x : 0.2697724997987128 et on trouve k : 0.3846862799943293 .

 27

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : 0.33

...

on trouve x : 0.4613060212717082 et on trouve k : 0.35867443944960586 .

 35

 et pour la distance suivante :  593.5037797672012 ,
on trouve x : 0.5025615073671406 et on trouve k : 0.3516135529022041 .

 36

 et pour la distance suivante :  1060.632920780966 ,
on trouve x : 1.0869389179979119 et on trouve k : 0.22098584286870332 .

 37

 et pour la distance suivante :  3916.2482599750642 ,
on trouve x : 4.659306709170644 et on trouve k : 7.706025225588727e-06 .

 38

 et pour la distance suivante :  14.18292671796798 ,
on trouve x : -0.2221674600756008 et on trouve k : 0.3892172050859124 .

 39

 et pour la distance suivante :  132.5339799632454 ,
on trouve x : -0.07411058250464704 et on trouve k : 0.39784821235826995 .

 40

 et pour la distance suivante :  1135.220161389213 ,
on trouve x : 1.180247373210423 et on trouve k : 0.19880507355404356 .

 41

 et pour la distance suivante :  308.49889550216227 ,
on trouve x : 0.146021095603323 et on trouve k : 0.3947117160255896 .

 42

 et 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 ,
on trouve x : -0.6391065172813369 et on trouve k : 0.3252480674252445 .

 18

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -0.5824642194534501 et on trouve k : 0.3366973522483034 .

 19

 et pour la distance suivante :  472.7788834473663 ,
on trouve x : -1.4745462816714658 et on trouve k : 0.1345146852883728 .

 20

 et pour la distance suivante :  4087.3026250595076 ,
on trouve x : 3.047214061095309 et on trouve k : 0.003842257093582704 .

 21

 et pour la distance suivante :  274.0527881314529 ,
on trouve x : -1.7231521399004117 et on trouve k : 0.09039510314272468 .

 22

 et pour la distance suivante :  842.1817635940587 ,
on trouve x : -1.0124241838909889 et on trouve k : 0.23896458994138925 .

 23

 et pour la distance suivante :  843.4354647082657 ,
on trouve x : -1.0108558068695184 et on trouve k : 0.23934403997151565 .

 24

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : -0.9853055280337215 et on trouve k : 0.24552609969585643 .

 

on trouve x : -4.024698192000252 et on trouve k : 0.00012120390061562187 .

 13

 et pour la distance suivante :  1151.0275805838955 ,
on trouve x : -3.3260410075702125 et on trouve k : 0.0015801856715250307 .

 14

 et pour la distance suivante :  1153.98358029554 ,
on trouve x : -3.3223430591751133 et on trouve k : 0.0015997302869069417 .

 15

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -3.2824464033090535 et on trouve k : 0.0018250174533701064 .

 16

 et pour la distance suivante :  515.7815382284228 ,
on trouve x : -4.120732249781106 et on trouve k : 8.197012794720667e-05 .

 17

 et pour la distance suivante :  1140.597749147637 ,
on trouve x : -3.33908870113694 et on trouve k : 0.0015129480050462558 .

 18

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -3.2824464033090535 et on trouve k : 0.0018250174533701064 .

 19

 et pour la distance suivante :  472.7788834473663 ,
on trouve x : -4.174528465527069 et on trouve k : 6.55771997016

6.859166296408174e-06

## 4. Influence sociale

J'ai choisi de ne pas implémenter cette seconde probabilité liée à l'influence sociale, par manque de temps.
Par la suite, je mettrais lui donne la valeur 0.

In [ ]:
Pui = 0

## 5. Génération et évaluation des recommandations

**5.1. Score de pertinence S pour un utilisateur u et un POI i**

Le "UserID et la liste Lu des "POIs visités par cet utilisateur" peut être vu comme faisant parti d'un même objet.
En effet, le UserID est défini non plus par son identifiant, mais par la liste des sites visités.

Par la suite, nous n'avons pas besoin de connaître l'userID, mais plutôt la liste des Lu pour déterminer la probabilité que cet user va visiter ou pas le POIx considéré.

UserID == Lu

In [ ]:
# u : utilisateur
# Lu = liste des POIs déjà visités par l'utilisateur u
# POIx = le POI considéré

# Etant donné que je n'ai calculé que l'influence géographique, le score de pertinence S est uniquement la probabilité calculée pour l'influence géographique.
# S = (Pui + geo_proba(Lu, POIx) )/2
S = geo_proba(Lu, POIx)

# print(S)

Du :  [364.1072348590375, 464.23430029528936, 595.0724660273181, 1051.997995877385, 1059.162046827403, 1124.8984168232785, 169.42099473532667, 1140.0487470689638, 1124.8984168232785, 629.2354239512177, 3723.868584535088, 332.0035950255078, 592.5475221681768, 1151.0275805838955, 1153.98358029554, 1185.8754540144741, 515.7815382284228, 1140.597749147637, 1185.8754540144741, 472.7788834473663, 4087.3026250595076, 274.0527881314529, 842.1817635940587, 843.4354647082657, 863.8593886794971, 630.2705167052557, 809.1496856623068, 863.8593886794971, 804.7816678089818, 4044.0109909280864, 568.1904418251492, 569.3827580883359, 593.5037797672012, 733.1785883224403, 560.5257086931276, 593.5037797672012, 1060.632920780966, 3916.2482599750642, 14.18292671796798, 132.5339799632454, 1135.220161389213, 308.49889550216227, 132.5339799632454, 1607.1541420475364, 3688.4293038481796, 118.35143867457997, 1143.9580515909122, 295.3331902843261, 118.35143867457997, 1611.3404042257596, 3701.414248583025, 1221.54

on trouve x : 0.12090013726503748 et on trouve k : 0.3960372704731312 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -1.1673348960535863 et on trouve k : 0.20184122349167422 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -1.4072451626957503 et on trouve k : 0.14821253216626315 .

 54

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : 0.6587463435452913 et on trouve k : 0.32112914824898264 .

 55

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : 3.3587285274008947 et on trouve k : 0.0014166403043333103 .

 56

 et pour la distance suivante :  1257.6979224220986 ,
on trouve x : 0.16613185605690645 et on trouve k : 0.39347473068272226 .

 57

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 0.12090013726503748 et on trouve k : 0.3960372704731312 .

 58

 et pour la distance suivante :  679.0232609634891 ,
on trouve x : -0.5577887272895643 et on trouve k : 0.3414675367161507 .

 59

 et pour l

on trouve x : -0.052438567706328015 et on trouve k : 0.39839415088372687 .

 47

 et pour la distance suivante :  295.3331902843261 ,
on trouve x : -1.114066189504873 et on trouve k : 0.21448611588805255 .

 48

 et pour la distance suivante :  118.35143867457997 ,
on trouve x : -1.3354699270455248 et on trouve k : 0.16354313574964371 .

 49

 et pour la distance suivante :  1611.3404042257596 ,
on trouve x : 0.5322556100418805 et on trouve k : 0.3462526561165785 .

 50

 et pour la distance suivante :  3701.414248583025 ,
on trouve x : 3.1469328672601575 et on trouve k : 0.002821372617311907 .

 51

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 0.04461801317319634 et on trouve k : 0.39854537738635326 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -1.2436170201454273 et on trouve k : 0.18410847084588466 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -1.4835272867875915 et on trouve k : 0.13273970966316084 .

 54

 et pour

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 ,
on trouve x : -0.1510265494397289 et on trouve k : 0.39441838488405573 .

 20

 et pour la distance suivante :  4087.3026250595076 ,
on trouve x : 4.370733793327046 et on trouve k : 2.8358167351791545e-05 .

 21

 et pour la distance suivante :  274.0527881314529 ,
on trouve x : -0.39963240766867486 et on trouve k : 0.36832426871151785 .

 22

 et pour la distance suivante :  842.1817635940587 ,
on trouve x : 0.3110955483407481 et on trouve k : 0.3800970161064281 .

 23

 et pour la distance suivante :  843.4354647082657 ,
on trouve x : 0.31266392536221854 et on trouve k : 0.3799111390107755 .

 24

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : 0.33821420419801534 et on trouve k : 0.37676525277261896 .

 25

 et pour la distance suivante :  630.2705167052557 ,
on trouve x : 0.04599509780651832 et on trouve k : 0.39852051253279414 .

 26

 et pour la distance suivante :  809.1496856623068 ,
on trouve x : 0.2697724997987128 et on trouve k : 0.3846862799943293 .

 

...

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : 0.840775711565156 et on trouve k : 0.28016111466387633 .

 25

 et pour la distance suivante :  630.2705167052557 ,
on trouve x : 0.5485566051736589 et on trouve k : 0.3432158574500323 .

 26

 et pour la distance suivante :  809.1496856623068 ,
on trouve x : 0.7723340071658534 et on trouve k : 0.2960613914792558 .

 27

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : 0.840775711565156 et on trouve k : 0.28016111466387633 .

 28

 et pour la distance suivante :  804.7816678089818 ,
on trouve x : 0.7668696275358962 et on trouve k : 0.29730906866679874 .

 29

 et pour la distance suivante :  4044.0109909280864 ,
on trouve x : 4.819137572489109 et on trouve k : 3.612964236195541e-06 .

 30

 et pour la distance suivante :  568.1904418251492 ,
on trouve x : 0.47089458363616715 et on trouve k : 0.3570750169895691 .

 31

 et pour la distance suivante :  569.3827580883359 ,
on trouve x : 0.4723861683594449

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 -0.7409823813122524 et on trouve k : 0.3031686650811653 .

 5

 et pour la distance suivante :  1124.8984168232785 ,
on trouve x : -0.6587463435452913 et on trouve k : 0.32112914824898264 .

 6

 et pour la distance suivante :  169.42099473532667 ,
on trouve x : -1.8540462570213994 et on trouve k : 0.07152685575834913 .

 7

 et pour la distance suivante :  1140.0487470689638 ,
on trouve x : -0.6397933175363363 et on trouve k : 0.32510525817270797 .

 8

 et pour la distance suivante :  1124.8984168232785 ,
on trouve x : -0.6587463435452913 et on trouve k : 0.32112914824898264 .

 9

 et pour la distance suivante :  629.2354239512177 ,
on trouve x : -1.2788195329238534 et on trouve k : 0.17611321352199258 .

 10

 et pour la distance suivante :  3723.868584535088 ,
on trouve x : 2.5925589670546634 et on trouve k : 0.013847929778019514 .

 11

 et pour la distance suivante :  332.0035950255078 ,
on trouve x : -1.6506558224940144 et on trouve k : 0.10215430084529732 .

 12

 et pour la 

 et pour la distance suivante :  364.1072348590375 ,
on trouve x : -4.310476431593167 et on trouve k : 3.6835737111261435e-05 .

 1

 et pour la distance suivante :  464.23430029528936 ,
on trouve x : -4.185217718110398 et on trouve k : 6.271171992134406e-05 .

 2

 et pour la distance suivante :  595.0724660273181 ,
on trouve x : -4.02153949342026 et on trouve k : 0.0001227539660571018 .

 3

 et pour la distance suivante :  1051.997995877385 ,
on trouve x : -3.449926775349633 et on trouve k : 0.0010385436218689529 .

 4

 et pour la distance suivante :  1059.162046827403 ,
on trouve x : -3.4409645651678553 et on trouve k : 0.0010711128732197562 .

 5

 et pour la distance suivante :  1124.8984168232785 ,
on trouve x : -3.3587285274008947 et on trouve k : 0.0014166403043333103 .

 6

 et pour la distance suivante :  169.42099473532667 ,
on trouve x : -4.554028440877003 et on trouve k : 1.2515708115197427e-05 .

 7

 et pour la distance suivante :  1140.0487470689638 ,
on trouve x : -3

on trouve x : -2.7480827111912105 et on trouve k : 0.009141618531441383 .

 62

 et pour la distance suivante :  3996.673067640781 ,
on trouve x : 0.23385452301164308 et on trouve k : 0.3881814080239486 .

 63

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : -2.699982183855603 et on trouve k : 0.01042143610920001 .

 64

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : 0.0 et on trouve k : 0.3989422804014327 .

 65

 et pour la distance suivante :  4195.0904356330475 ,
on trouve x : 0.48207416411529425 et on trouve k : 0.35517797288600655 .
d'où au final sommeK :  4.342502277200198 et Fij  8.23099955568981e-05

 La probabilité pour le point POIx est  6.859166296408174e-06


**5.2. Algorithme de recommandation personnalisée**

Nous allons ici créer notre propre algorithme dans Surprise, basé sur ce score de pertinence S.

----

Nous avons un dataset avec pour chaque ligne : un utilisateur donné et la liste des POIs qu'il a visité.

Avec la fonction précédente "geo_proba", nous sommes capables de calculer la probabilité pour cet utilisateur, de visiter un nouveau POIx ==> que nous rajouterons à une dernière colonne grâce à la création de notre algorithme personnalisée.

Nous aboutissons à un dataset avec 3 colonnes :
- col 1 : liste des POIs déjà visités par un userid
- col 2 : le POI pour lequel on va estimer la probabilité de visite par cet userid
- col 3 : probabilité calculé de visiter le POIx, pour cet userid

Nous avons donc un premier modèle de prédiction basée sur un calcul de probabilité (ici uniquement géographique).

Nous allons calculer :
     pour chaque utilisateur (donc chaque liste de POIs visités par cet utilisateur)
          pour chaque POI à considérer
               la probabilité que cet utilisateur visite le POI considéré
==> nous aurons ainsi les scores des POIs et il suffira de prendre le POI qui maximise la probabilité
==> cela correspond à la recommandation


**Cas simpliste :**

Au vu de notre puissance de calcul limitée, nous allons considérons un exemple simple.

- un unique utilisateur, par exemple userid = 3
- la liste des POIs que ce userid a visité, et par soucis de limiter le temps de calcul, prenons cette liste simplifiée 
Lu = [9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, 420315, 15245, 18758]
- la liste des POIs sur lesquels nous allons calculer la probabilité de visite : List_POIx = [8947, 8995, 9074, 9184, 9186, 9197, 9220, 420315]

In [ ]:
Lu = [9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, 420315, 15245, 18758]
List_POIx = [8947, 8995, 9074, 9184, 9186, 9197, 9220, 420315]

index_proba = []

for value in List_POIx :
    S = geo_proba(Lu, value)
    index_proba.append(S)


Du :  [364.1072348590375, 464.23430029528936, 595.0724660273181, 1051.997995877385, 1059.162046827403, 1124.8984168232785, 169.42099473532667, 1140.0487470689638, 1124.8984168232785, 629.2354239512177, 3723.868584535088, 332.0035950255078, 592.5475221681768, 1151.0275805838955, 1153.98358029554, 1185.8754540144741, 515.7815382284228, 1140.597749147637, 1185.8754540144741, 472.7788834473663, 4087.3026250595076, 274.0527881314529, 842.1817635940587, 843.4354647082657, 863.8593886794971, 630.2705167052557, 809.1496856623068, 863.8593886794971, 804.7816678089818, 4044.0109909280864, 568.1904418251492, 569.3827580883359, 593.5037797672012, 733.1785883224403, 560.5257086931276, 593.5037797672012, 1060.632920780966, 3916.2482599750642, 14.18292671796798, 132.5339799632454, 1135.220161389213, 308.49889550216227, 132.5339799632454, 1607.1541420475364, 3688.4293038481796, 118.35143867457997, 1143.9580515909122, 295.3331902843261, 118.35143867457997, 1611.3404042257596, 3701.414248583025, 1221.54

on trouve x : -2968.8902552951713 et on trouve k : 0.0 .

 59

 et pour la distance suivante :  3587.362809229284 ,
on trouve x : -2965.2519296476594 et on trouve k : 0.0 .

 60

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -2969.4998014639355 et on trouve k : 0.0 .

 61

 et pour la distance suivante :  1613.0255250935695 ,
on trouve x : -2967.721820751672 et on trouve k : 0.0 .

 62

 et pour la distance suivante :  3996.673067640781 ,
on trouve x : -2964.739883517469 et on trouve k : 0.0 .

 63

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : -2967.6737202243366 et on trouve k : 0.0 .

 64

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : -2964.9737380404813 et on trouve k : 0.0 .

 65

 et pour la distance suivante :  4195.0904356330475 ,
on trouve x : -2964.4916638763657 et on trouve k : 0.0 .
d'où au final sommeK :  0.0 et Fij  0.0

 calcul de f pour le point fixe :  2498079

 0

 et pour la distance suivante :  364.107

on trouve x : -2964.8001303903397 et on trouve k : 0.0 .

 63

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : -2967.733967097208 et on trouve k : 0.0 .

 64

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : -2965.033984913352 et on trouve k : 0.0 .

 65

 et pour la distance suivante :  4195.0904356330475 ,
on trouve x : -2964.5519107492364 et on trouve k : 0.0 .
d'où au final sommeK :  0.0 et Fij  0.0

 calcul de f pour le point fixe :  7062725

 0

 et pour la distance suivante :  364.1072348590375 ,
on trouve x : -2968.929205633326 et on trouve k : 0.0 .

 1

 et pour la distance suivante :  464.23430029528936 ,
on trouve x : -2968.803946919843 et on trouve k : 0.0 .

 2

 et pour la distance suivante :  595.0724660273181 ,
on trouve x : -2968.640268695153 et on trouve k : 0.0 .

 3

 et pour la distance suivante :  1051.997995877385 ,
on trouve x : -2968.0686559770825 et on trouve k : 0.0 .

 4

 et pour la distance suivante :  1059.1620468274

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 0.0 .

 20

 et pour la distance suivante :  4087.3026250595076 ,
on trouve x : -2963.9564521959473 et on trouve k : 0.0 .

 21

 et pour la distance suivante :  274.0527881314529 ,
on trouve x : -2968.726818396943 et on trouve k : 0.0 .

 22

 et pour la distance suivante :  842.1817635940587 ,
on trouve x : -2968.0160904409336 et on trouve k : 0.0 .

 23

 et pour la distance suivante :  843.4354647082657 ,
on trouve x : -2968.0145220639124 et on trouve k : 0.0 .

 24

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : -2967.9889717850765 et on trouve k : 0.0 .

 25

 et pour la distance suivante :  630.2705167052557 ,
on trouve x : -2968.2811908914678 et on trouve k : 0.0 .

 26

 et pour la distance suivante :  809.1496856623068 ,
on trouve x : -2968.057413489476 et on trouve k : 0.0 .

 27

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : -2967.9889717850765 et on trouve k : 0.0 .

 28

 et pour la distance suivante :  804.7816678089818 ,
on trou

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





 et pour la distance suivante :  1611.3404042257596 ,
on trouve x : 1.775872630187308 et on trouve k : 0.08243045321194804 .

 50

 et pour la distance suivante :  3701.414248583025 ,
on trouve x : 4.390549887405585 et on trouve k : 2.6000294577496356e-05 .

 51

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 1.2882350333186237 et on trouve k : 0.1739976851243084 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : 0.0 et on trouve k : 0.3989422804014327 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -0.2399102666421641 et on trouve k : 0.387624961368013 .

 54

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : 1.8260812395988775 et on trouve k : 0.07530378498668278 .

 55

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : 4.526063423454481 et on trouve k : 1.4210014580665998e-05 .

 56

 et pour la distance suivante :  1257.6979224220986 ,
on trouve x : 1.3334667521104926 et on trouve k : 0.

 et pour la distance suivante :  118.35143867457997 ,
on trouve x : 0.14805735974206666 et on trouve k : 0.3945935528024599 .

 46

 et pour la distance suivante :  1143.9580515909122 ,
on trouve x : 1.4310887190812633 et on trouve k : 0.14328122176583072 .

 47

 et pour la distance suivante :  295.3331902843261 ,
on trouve x : 0.36946109728271864 et on trouve k : 0.37262255654809967 .

 48

 et pour la distance suivante :  118.35143867457997 ,
on trouve x : 0.14805735974206666 et on trouve k : 0.3945935528024599 .

 49

 et pour la distance suivante :  1611.3404042257596 ,
on trouve x : 2.0157828968294718 et on trouve k : 0.052306801070342177 .

 50

 et pour la distance suivante :  3701.414248583025 ,
on trouve x : 4.630460154047749 et on trouve k : 8.810915985427851e-06 .

 51

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 1.5281452999607879 et on trouve k : 0.1241142753826386 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : 0.239910

on trouve x : -0.6458338663884544 et on trouve k : 0.3238453393985333 .

 41

 et pour la distance suivante :  308.49889550216227 ,
on trouve x : -1.6800601439955545 et on trouve k : 0.09727244006557041 .

 42

 et pour la distance suivante :  132.5339799632454 ,
on trouve x : -1.9001918221035246 et on trouve k : 0.06559190345448682 .

 43

 et pour la distance suivante :  1607.1541420475364 ,
on trouve x : -0.05544561313739686 et on trouve k : 0.3983295340767476 .

 44

 et pour la distance suivante :  3688.4293038481796 ,
on trouve x : 2.5482245137651773 et on trouve k : 0.01551942801699058 .

 45

 et pour la distance suivante :  118.35143867457997 ,
on trouve x : -1.917934146498975 et on trouve k : 0.06340743022943235 .

 46

 et pour la distance suivante :  1143.9580515909122 ,
on trouve x : -0.6349027871597781 et on trouve k : 0.32612018243520774 .

 47

 et pour la distance suivante :  295.3331902843261 ,
on trouve x : -1.6965304089583229 et on trouve k : 0.09460487727219118 .



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(index_proba)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.6353138228770655e-06, 6.859166296408174e-06]


On en déduit que parmi les 8 POIx proposés, ce sont les 2 derniers qui ont une proba non nulle pour l'userid 3.

Pour rappel, List_POIx = [8947, 8995, 9074, 9184, 9186, 9197, 9220, 420315].

Le POI de référence 420315 obtient le meilleur score, puis en seconde position le POI de référence 9220.

Pour rappel, la liste des POIs déjà visité par l'userid 3 est Lu = [9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, **420315**, 15245, 18758] ==> Il est donc normal (rassurant) d'aboutir à 420315 comment un POI à recommander !


In [ ]:
class GSLR(AlgoBase):

    def __init__(self, Lu, value):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.S = geo_proba(Lu, value)
        return self

    def estimate(self, u, i):
        return self.S


Essayons tout d'abord de comprendre ce qu'est la précision et le recall.

**Définition**

- Precision@N est défini de la façon suivante :
ensemble de recommandations "réellement bonnes" / ensemble des recommandations "prédictes bonnes"
= la proportion des items pertinents parmi l'ensemble des items proposés 

- Recall@N est défini de la façon suivante :
ensemble de recommandations "réellement bonnes" / ensemble des "vraies bonnes"
= la proportion des items pertinents proposés parmi l'ensemble des items pertinents

Etant donné que le rating est un nombre compris entre 0 et 1, il faut pouvoir le rendre binaire : OK ou NOK. Pour cela, on introduit un seuil. Si le rating est supérieur à ce seuil, alors c'est OK, sinon NOK.

Le nombre N représente le nbre de données à prendre en considération dans le calcul. On prend les N meilleurs scores (les plus élevés).


**Exemple**

Imaginons le cas suivant :

Un user a donné les notes suivantes (1ère colonne) et le système de recommandation prédit les notes suivantes (2nde colonne) :

| user| pred|
|-----|-----|
| 0.1 | 0.1 |
| 0.8 | 0.1 |
| 0.9 | 0.6 |
| 0.6 | 0.4 |
| 0.4 | 0.4 |
| 0.1 | 0.8 |
| 0.3 | 0.5 |
| 0.6 | 0.5 |
| 0.8 | 0.5 |
| 0.5 | 0.4 |
| 0.8 | 0.6 |


Prenons N = 4, et un seuil = 0.6 ==> Cela se traduit par :

| user| pred|
|-----|-----|
| NOK | NOK |
| OK  | NOK |
| OK  | OK  |
| NOK | NOK |
| NOK | NOK |
| NOK | OK  |
| NOK | NOK |
| NOK | NOK |
| OK  | NOK |
| NOK | NOK |
| OK  | OK  |

D'où les 2 indicateurs calculés suivants :

- Precision = toutes les prédictions OK qui sont réellement OK / toutes les prédictions OK = 2 / 3
- Recall = toutes les prédictions OK qui sont réellement OK / tous les vrais OK = 2 / 4



**Application sur notre problème actuel**

Prenons l'userid 3.
On sait que cet userid 3 a visité les POIs suivants : Lu.
On va calculer la probabilité de visiter les points d'intérêt du Lu mais également d'autres POIs. On espère que notre système va pouvoir prédire une grande probabilité pour les POIs du Lu, et des très faibles probabilités pour les points non Lu.


| POI   | user| pred|
|-------|-----|-----|
|   9246| OK  | ??? |
|2498079| OK  | ??? |
|7062725| OK  | ??? |
|7048639| OK  | ??? |
|  21714| OK  | ??? |
|   9371| OK  | ??? |
| 420315| OK  | ??? |
|6886818| OK  | ??? |
|  18328| OK  | ??? |
| 420315| OK  | ??? |
|  15245| OK  | ??? |
|  18758| OK  | ??? |
| 789570| NOK | ??? |
|7540930| NOK | ??? |
|7569147| NOK | ??? |
|6906866| NOK | ??? |

In [ ]:
# on calcule la probabilité pour chaque POI de la liste ci-dessus (colonne 1).

Lu = [9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, 420315, 15245, 18758]
List_POIx = [9246, 2498079, 7062725, 7048639, 21714, 9371, 420315, 6886818, 18328, 420315, 15245, 18758, 789570, 7540930, 7569147, 6906866]

index_proba = []

for value in List_POIx :
    S = geo_proba(Lu, value)
    index_proba.append(S)
    
# 5 min de traitement sur mon ordinateur

Du :  [364.1072348590375, 464.23430029528936, 595.0724660273181, 1051.997995877385, 1059.162046827403, 1124.8984168232785, 169.42099473532667, 1140.0487470689638, 1124.8984168232785, 629.2354239512177, 3723.868584535088, 332.0035950255078, 592.5475221681768, 1151.0275805838955, 1153.98358029554, 1185.8754540144741, 515.7815382284228, 1140.597749147637, 1185.8754540144741, 472.7788834473663, 4087.3026250595076, 274.0527881314529, 842.1817635940587, 843.4354647082657, 863.8593886794971, 630.2705167052557, 809.1496856623068, 863.8593886794971, 804.7816678089818, 4044.0109909280864, 568.1904418251492, 569.3827580883359, 593.5037797672012, 733.1785883224403, 560.5257086931276, 593.5037797672012, 1060.632920780966, 3916.2482599750642, 14.18292671796798, 132.5339799632454, 1135.220161389213, 308.49889550216227, 132.5339799632454, 1607.1541420475364, 3688.4293038481796, 118.35143867457997, 1143.9580515909122, 295.3331902843261, 118.35143867457997, 1611.3404042257596, 3701.414248583025, 1221.54

on trouve x : 1.5281452999607879 et on trouve k : 0.1241142753826386 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : 0.2399102666421641 et on trouve k : 0.387624961368013 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : 0.0 et on trouve k : 0.3989422804014327 .

 54

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : 2.0659915062410414 et on trouve k : 0.04721238696438449 .

 55

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : 4.7659736900966445 et on trouve k : 4.66141671213063e-06 .

 56

 et pour la distance suivante :  1257.6979224220986 ,
on trouve x : 1.5733770187526568 et on trouve k : 0.11570676749187771 .

 57

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 1.5281452999607879 et on trouve k : 0.1241142753826386 .

 58

 et pour la distance suivante :  679.0232609634891 ,
on trouve x : 0.849456435406186 et on trouve k : 0.2781133120509434 .

 59

 et pour la distance suivante :  358

 et pour la distance suivante :  295.3331902843261 ,
on trouve x : -0.08603616122075929 et on trouve k : 0.3974684799625672 .

 48

 et pour la distance suivante :  118.35143867457997 ,
on trouve x : -0.30743989876141126 et on trouve k : 0.38052698708258265 .

 49

 et pour la distance suivante :  1611.3404042257596 ,
on trouve x : 1.560285638325994 et on trouve k : 0.11810465157713651 .

 50

 et pour la distance suivante :  3701.414248583025 ,
on trouve x : 4.1749628955442715 et on trouve k : 6.545837439314762e-05 .

 51

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : 1.0726480414573099 et on trouve k : 0.2244223652016938 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -0.21558699186131383 et on trouve k : 0.3897782042306445 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -0.45549725850347794 et on trouve k : 0.3596307711837544 .

 54

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : 1.6104942477375637 et

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1185.8754540144741 ,
on trouve x : 0.7390930901112064 et on trouve k : 0.30359283583219754 .

 19

 et pour la distance suivante :  472.7788834473663 ,
on trouve x : -0.15298897210680928 et on trouve k : 0.39430074605602516 .

 20

 et pour la distance suivante :  4087.3026250595076 ,
on trouve x : 4.368771370659966 et on trouve k : 2.8602392844297075e-05 .

 21

 et pour la distance suivante :  274.0527881314529 ,
on trouve x : -0.4015948303357552 et on trouve k : 0.36803481582032527 .

 22

 et pour la distance suivante :  842.1817635940587 ,
on trouve x : 0.30913312567366774 et on trouve k : 0.38032840420310027 .

 23

 et pour la distance suivante :  843.4354647082657 ,
on trouve x : 0.3107015026951382 et on trouve k : 0.38014358396399045 .

 24

 et pour la distance suivante :  863.8593886794971 ,
on trouve x : 0.336251781530935 et on trouve k : 0.37701467615601497 .

 25

 et pour la distance suivante :  630.2705167052557 ,
on trouve x : 0.044032675139437955 et on trouve k : 0.39

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 1611.3404042257596 ,
on trouve x : -18450.5874091864 et on trouve k : 0.0 .

 50

 et pour la distance suivante :  3701.414248583025 ,
on trouve x : -18447.97273192918 et on trouve k : 0.0 .

 51

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : -18451.07504678327 et on trouve k : 0.0 .

 52

 et pour la distance suivante :  191.77516915989466 ,
on trouve x : -18452.363281816586 et on trouve k : 0.0 .

 53

 et pour la distance suivante :  0.0 ,
on trouve x : -18452.60319208323 et on trouve k : 0.0 .

 54

 et pour la distance suivante :  1651.475262554055 ,
on trouve x : -18450.53720057699 et on trouve k : 0.0 .

 55

 et pour la distance suivante :  3809.7386302902687 ,
on trouve x : -18447.83721839313 et on trouve k : 0.0 .

 56

 et pour la distance suivante :  1257.6979224220986 ,
on trouve x : -18451.029815064478 et on trouve k : 0.0 .

 57

 et pour la distance suivante :  1221.5414017191245 ,
on trouve x : -18451.07504678327 et on trouve k : 0.0 .

 58

 et 

 et pour la distance suivante :  3723.868584535088 ,
on trouve x : -18447.971128964982 et on trouve k : 0.0 .

 11

 et pour la distance suivante :  332.0035950255078 ,
on trouve x : -18452.21434375453 et on trouve k : 0.0 .

 12

 et pour la distance suivante :  592.5475221681768 ,
on trouve x : -18451.888403940182 et on trouve k : 0.0 .

 13

 et pour la distance suivante :  1151.0275805838955 ,
on trouve x : -18451.189746755754 et on trouve k : 0.0 .

 14

 et pour la distance suivante :  1153.98358029554 ,
on trouve x : -18451.186048807358 et on trouve k : 0.0 .

 15

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -18451.146152151494 et on trouve k : 0.0 .

 16

 et pour la distance suivante :  515.7815382284228 ,
on trouve x : -18451.984437997966 et on trouve k : 0.0 .

 17

 et pour la distance suivante :  1140.597749147637 ,
on trouve x : -18451.20279444932 et on trouve k : 0.0 .

 18

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -1845

 et pour la distance suivante :  1153.98358029554 ,
on trouve x : -18453.197901611118 et on trouve k : 0.0 .

 15

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -18453.15800495525 et on trouve k : 0.0 .

 16

 et pour la distance suivante :  515.7815382284228 ,
on trouve x : -18453.996290801722 et on trouve k : 0.0 .

 17

 et pour la distance suivante :  1140.597749147637 ,
on trouve x : -18453.21464725308 et on trouve k : 0.0 .

 18

 et pour la distance suivante :  1185.8754540144741 ,
on trouve x : -18453.15800495525 et on trouve k : 0.0 .

 19

 et pour la distance suivante :  472.7788834473663 ,
on trouve x : -18454.05008701747 et on trouve k : 0.0 .

 20

 et pour la distance suivante :  4087.3026250595076 ,
on trouve x : -18449.528326674703 et on trouve k : 0.0 .

 21

 et pour la distance suivante :  274.0527881314529 ,
on trouve x : -18454.298692875698 et on trouve k : 0.0 .

 22

 et pour la distance suivante :  842.1817635940587 ,
on trouve x : -18453.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(index_proba)

[6.818593656003889e-06, 6.55748026134767e-06, 6.648793824651961e-06, 6.8233565812703675e-06, 6.7851550966276036e-06, 6.7984794189574e-06, 6.859166296408174e-06, 6.642790265639782e-06, 6.730210515040128e-06, 6.859166296408174e-06, 6.263790325944505e-06, 2.1427621978385906e-05, 7.378136099280205e-108, 0.0, 0.0, 0.0]


On voit que nos valeurs sont très très faibles.

Pour les exploiter, nous allons les élever en multipliant par 100000.

In [ ]:
index_proba2 = [i*100000 for i in index_proba]
print(index_proba2)

[0.6818593656003888, 0.655748026134767, 0.6648793824651961, 0.6823356581270368, 0.6785155096627603, 0.67984794189574, 0.6859166296408175, 0.6642790265639782, 0.6730210515040128, 0.6859166296408175, 0.6263790325944505, 2.1427621978385907, 7.378136099280205e-103, 0.0, 0.0, 0.0]


On va choisir un seuil pour lequel on dit que la prédition est OK ou pas : prenons 0.5 comme seuil.

On va choisir un nbre K des meilleures scores : prenons K = 5


In [ ]:
# étape 1 : commençons par filtrer les proba pour ne garder que les valeurs qui dépassent le seuil

seuil = 0.5

index_proba3 = []
init = 0

for i in index_proba2 :
    
    # déterminer si la valeur est supérieure au seuil ou pas
    if i>= seuil : 
        index_proba3.append(i)
    else :
        index_proba3.append(0)

print("proba3 seuillé : ", index_proba3)


# étape 2 : recherchons les K meilleures proba

K=5

K_meilleures_valeurs = list(index_proba3)
K_meilleures_valeurs.sort(reverse=True)
K_meilleures_valeurs=K_meilleures_valeurs[:K]
print("K_meilleures_valeurs : ", K_meilleures_valeurs)


# étape 3 : recherons les index dans la liste des probas calculés, qui correspondent à ces K meilleurs proba

index = []

for k in K_meilleures_valeurs :

    for i, e in enumerate(index_proba3):
        #print(e)
        if e == k :
            index.append(i)
            print("la valeur recherchée est ", k, "la valeur regardée est ", e, "l'indice identifiée est ", i)

print(index)
          
    
# étape 4 : maintenant, ne gardons que ces K meilleurs valeurs dans notre liste des proba calculées

pred = ["NOK"] * len(index_proba3)
#print("liste vide ", pred)

for ind in index :
    pred[ind]="OK"

print("pred : ", pred)


proba3 seuillé :  [0.6818593656003888, 0.655748026134767, 0.6648793824651961, 0.6823356581270368, 0.6785155096627603, 0.67984794189574, 0.6859166296408175, 0.6642790265639782, 0.6730210515040128, 0.6859166296408175, 0.6263790325944505, 2.1427621978385907, 0, 0, 0, 0]
K_meilleures_valeurs :  [2.1427621978385907, 0.6859166296408175, 0.6859166296408175, 0.6823356581270368, 0.6818593656003888]
la valeur recherchée est  2.1427621978385907 la valeur regardée est  2.1427621978385907 l'indice identifiée est  11
la valeur recherchée est  0.6859166296408175 la valeur regardée est  0.6859166296408175 l'indice identifiée est  6
la valeur recherchée est  0.6859166296408175 la valeur regardée est  0.6859166296408175 l'indice identifiée est  9
la valeur recherchée est  0.6859166296408175 la valeur regardée est  0.6859166296408175 l'indice identifiée est  6
la valeur recherchée est  0.6859166296408175 la valeur regardée est  0.6859166296408175 l'indice identifiée est  9
la valeur recherchée est  0.682

In [ ]:
# Pour pouvoir calculer la précision et le recall, il faut d'abord définir les valeurs réelles de la préférence du user
user = ['OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'NOK', 'NOK', 'NOK', 'NOK']


In [ ]:
# calcul de la précision
# rappel : Precision = toutes les prédictions OK qui sont réellement OK / toutes les prédictions OK

pred_reellementOK = 0
pred_ok = 0

for i in range(0, len(pred)) :
    if pred[i] == user[i]:
        pred_reellementOK += 1
    if pred[i] == 'OK':
        pred_ok += 1

precision=pred_reellementOK / pred_ok
print("La précision pour un seuil ", seuil, " et pour une valeur K de ", K, " est ", precision)

La précision pour un seuil  0.5  et pour une valeur K de  5  est  1.8


In [ ]:
# calcul du recall
# rappel : Recall = toutes les prédictions OK qui sont réellement OK / tous les vrais OK

pred_reellementOK = 0
user_ok = 0

for i in range(0, len(pred)) :
    if pred[i] == user[i]:
        pred_reellementOK += 1
    if user[i] == 'OK':
        user_ok += 1

recall=pred_reellementOK / user_ok
print("Le recall pour un seuil ", seuil, " et pour une valeur K de ", K, " est ", recall)

Le recall pour un seuil  0.5  et pour une valeur K de  5  est  0.75


Je termine ici mon TP.

J'ai décidé de ne pas calculer les probabilités liées à des interactions sociales (ami d'un ami), par manque de temps.
J'ai essayé de comprendre les principes d'un système de recommandation, et de les appliquer en utilisant la librairie Surprise (il y a peu d'informations disponibles ou tutorials sur la toile).

J'espère que ce TP vous conviendra.

Bien à vous.

Hy-Boui